In [1]:
import pandas as pd
import io
import datetime as dt
from js import fetch
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
url='https://raw.githubusercontent.com/Shreedhar-Pokhrel/Projects/main/data/'
req_doc = ["10-21_1.csv","10-21_2.csv","after22.csv"]
# joining data gathered from github
dfs = [await (await fetch(url+x)).text() for x in req_doc]
dfs = [pd.read_csv(io.StringIO(x),low_memory=False,parse_dates=["date"]).iloc[:,1:].dropna(how="all") for x in dfs]
dfs = pd.concat(dfs)
dfs = dfs[dfs.loc[:,"sector"]!="Not Found"]
dfs = dfs[dfs.loc[:,"closing price"]!=0.0]
dfs["diff"]=dfs.loc[:,"closing price"]-dfs.loc[:,"previous closing"]
dfs["vwap"] = (dfs.loc[:,"amount"]/dfs.loc[:,"traded shares"]).round(2)
dfs["range"]=dfs.loc[:,"max price"]-dfs.loc[:,"min price"]
for i in ["diff","range"]:
    dfs[i+"%"] = (dfs[i]*100/dfs["closing price"]).round(2)
stocks = list(dfs[dfs.loc[:,"date"]==dfs.loc[:,"date"].max()].loc[:,"stock symbol"])
sectors = list(dfs[dfs.loc[:,"date"]==dfs.loc[:,"date"].max()].loc[:,"sector"].drop_duplicates())

In [3]:
def calculation(dfcc,column,basis):
    if basis =="min":
        result = dfcc.loc[:,column].min()
    elif basis == "max":
        result = dfcc.loc[:,column].max()
    elif basis == "mean":
        result = dfcc.loc[:,column].mean().round(2)
    elif basis == "sum":
        result = dfcc.loc[:,column].sum()
    else:
        print("basis not valid")
    return result

def find_average(dfs):
    price = "closing price"
    index= ["stock symbol","sector","closing price","vwap"]
    percent = ["diff","range"]
    base = index+percent
    days = [(120,price,"mean"),
            (180,price,"mean"),
            (365,price,"min"),
            (365,price,"max")]
    calc = ["120d","180d","52wl","52wh"]
    columns = base+calc
    
    latest = dfs.loc[:,"date"].max()
    dates = [(latest - dt.timedelta(x),y,z) for x,y,z in days]
    dfn= [(dfs[dfs.loc[:,"date"]>x],y,z) for x,y,z in dates]
    basedata = dfs[dfs.loc[:,"date"]==dfs.loc[:,"date"].max()].loc[:,base].to_numpy()

    averages =[]
    for i in range(0,len(basedata)):
        sym = basedata[i][0]
        dfa = [(x[x.loc[:,"stock symbol"]==sym],y,z) for x,y,z in dfn]
        results = [calculation(x,y,z) for x,y,z in dfa]
        averages.append(list(basedata[i])+results)
    new = pd.DataFrame(averages,columns=columns)
    
    for i in percent:
        new[i] = (new[i]*100/new[price]).round(2)
    for i in calc+["vwap"]:
        new[i+" diff"] = ((new[price]-new[i])*100/new[price]).round(2)
    return new
df_avg=find_average

def sortdf(bol,basis,col=10,sec=sectors,avg = df_avg):
    if len(avg)==0:
        print(len(avg),end=" ")
        print("Proc Empty")
        global df_avg
        df_avg=find_average(dfs)
        avg=proc
    # 1=True, 0=False
    watching = ["stock symbol","sector","closing price","diff","range","52wl","120d"]
    watch = list(dict.fromkeys(watching +[basis]))
    df_res = avg[avg.loc[:,"sector"].isin(sec)].sort_values(ascending=bol, by=basis).loc[:,watch].reset_index(drop=True).head(col)
    display(df_res)

In [ ]:
df_graph = dfs[dfs["sector"]=="Hydro Power"]
graph_sc = list(df_graph[df_graph.loc[:,"date"]==df_graph.loc[:,"date"].max()].loc[:,"stock symbol"])
for i in graph_sc:
    # plt.figure(figsize=(12,6))
    # plt.plot(df_graph[df_graph["stock symbol"]==i].iloc[-300:,:].loc[:,"range"].reset_index(drop=True))
    # plt.grid(True)
    # plt.title(i)
    # plt.show()
    df_graph[df_graph["stock symbol"]==i].iloc[-300:,:].loc[:,"range"].reset_index(drop=True).plot(
        figsize=(20,20),grid=True,legend=i)